In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'yeong_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-11-6 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-11-6 23:00
data/yeong/자동/용봉_2015.xlsx
data/ye

Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.5719 - disc_loss: 0.3277 - rmse: 0.5192 - val_loss: 0.4000
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.7659 - disc_loss: 0.3633 - rmse: 0.3981 - val_loss: 0.4817
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.0512 - disc_loss: 0.3633 - rmse: 0.4412 - val_loss: 0.4129
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.9607 - disc_loss: 0.3387 - rmse: 0.4398 - val_loss: 0.4411
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.2897 - disc_loss: 0.3480 - rmse: 0.4338 - val_loss: 0.4350
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.6675 - disc_loss: 0.3644 - rmse: 0.4887 - val_loss: 0.5126
Epoch 55/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.8348 - disc_loss: 0.3539 - rmse: 0.4527 - val_loss: 0.4581
Epoch 56/2000
1/1 [=

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1879 - disc_loss: 0.2712 - rmse: 0.3499 - val_loss: 0.3509
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5617 - disc_loss: 0.2798 - rmse: 0.3644 - val_loss: 0.3843
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4514 - disc_loss: 0.2835 - rmse: 0.3893 - val_loss: 0.3639
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0917 - disc_loss: 0.2636 - rmse: 0.3908 - val_loss: 0.3596
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5296 - disc_loss: 0.2721 - rmse: 0.3381 - val_loss: 0.3473
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0096 - disc_loss: 0.2603 - rmse: 0.3757 - val_loss: 0.3368
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2719 - disc_loss: 0.2661 - rmse: 0.3719 - val_loss: 0.3724
Epoch 114/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9089 - disc_loss: 0.2285 - rmse: 0.3165 - val_loss: 0.3424
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0051 - disc_loss: 0.2388 - rmse: 0.3358 - val_loss: 0.3416
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1194 - disc_loss: 0.2409 - rmse: 0.3130 - val_loss: 0.3442
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7372 - disc_loss: 0.2349 - rmse: 0.3182 - val_loss: 0.3261
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5657 - disc_loss: 0.2444 - rmse: 0.3155 - val_loss: 0.3654
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7534 - disc_loss: 0.2401 - rmse: 0.3263 - val_loss: 0.3486
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4054 - disc_loss: 0.2381 - rmse: 0.3686 - val_loss: 0.3211
Epoch 172/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3784 - disc_loss: 0.2269 - rmse: 0.3309 - val_loss: 0.3059
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3051 - disc_loss: 0.2333 - rmse: 0.3152 - val_loss: 0.3029
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2715 - disc_loss: 0.2233 - rmse: 0.2775 - val_loss: 0.3254
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3268 - disc_loss: 0.2294 - rmse: 0.3006 - val_loss: 0.4222
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2080 - disc_loss: 0.2268 - rmse: 0.3298 - val_loss: 0.3743
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5165 - disc_loss: 0.2434 - rmse: 0.3364 - val_loss: 0.3359
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5705 - disc_loss: 0.2276 - rmse: 0.2521 - val_loss: 0.3428
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2954 - disc_loss: 0.1706 - rmse: 0.2616 - val_loss: 0.2735
Epoch 1152/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0558 - disc_loss: 0.1648 - rmse: 0.2402 - val_loss: 0.2633
Epoch 1153/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5926 - disc_loss: 0.1677 - rmse: 0.3024 - val_loss: 0.2601
Epoch 1154/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3187 - disc_loss: 0.1655 - rmse: 0.2453 - val_loss: 0.2924
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4001 - disc_loss: 0.1715 - rmse: 0.2539 - val_loss: 0.2621
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6702 - disc_loss: 0.1669 - rmse: 0.2480 - val_loss: 0.3364
Epoch 1157/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8936 - disc_loss: 0.1616 - rmse: 0.2369 - val_loss: 0.2688
Epoch 1158/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7151 - disc_loss: 0.1687 - rmse: 0.2655 - val_loss: 0.2501
Epoch 1210/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2162 - disc_loss: 0.1543 - rmse: 0.2566 - val_loss: 0.3239
Epoch 1211/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0377 - disc_loss: 0.1704 - rmse: 0.3875 - val_loss: 0.2506
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1025 - disc_loss: 0.1647 - rmse: 0.2850 - val_loss: 0.2942
Epoch 1213/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9762 - disc_loss: 0.1542 - rmse: 0.2745 - val_loss: 0.3187
Epoch 1214/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9514 - disc_loss: 0.1608 - rmse: 0.2810 - val_loss: 0.3356
Epoch 1215/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6114 - disc_loss: 0.1672 - rmse: 0.3639 - val_loss: 0.3130
Epoch 1216/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2534 - disc_loss: 0.1645 - rmse: 0.3173 - val_loss: 0.2755
Epoch 1268/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8409 - disc_loss: 0.1668 - rmse: 0.2507 - val_loss: 0.2490
Epoch 1269/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8817 - disc_loss: 0.1512 - rmse: 0.2660 - val_loss: 0.2598
Epoch 1270/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3832 - disc_loss: 0.1678 - rmse: 0.2354 - val_loss: 0.2703
Epoch 1271/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5776 - disc_loss: 0.1714 - rmse: 0.2428 - val_loss: 0.2804
Epoch 1272/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4049 - disc_loss: 0.1747 - rmse: 0.2947 - val_loss: 0.2796
Epoch 1273/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6961 - disc_loss: 0.1718 - rmse: 0.3586 - val_loss: 0.2934
Epoch 1274/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3946 - disc_loss: 0.1626 - rmse: 0.2424 - val_loss: 0.3222
Epoch 1326/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8719 - disc_loss: 0.1635 - rmse: 0.2626 - val_loss: 0.3381
Epoch 1327/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9835 - disc_loss: 0.1670 - rmse: 0.2930 - val_loss: 0.3034
Epoch 1328/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5741 - disc_loss: 0.1665 - rmse: 0.2454 - val_loss: 0.3223
Epoch 1329/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3915 - disc_loss: 0.1586 - rmse: 0.2439 - val_loss: 0.2932
Epoch 1330/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4985 - disc_loss: 0.1595 - rmse: 0.2509 - val_loss: 0.3114
Epoch 1331/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8998 - disc_loss: 0.1480 - rmse: 0.2522 - val_loss: 0.2642
Epoch 1332/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2938 - disc_loss: 0.1609 - rmse: 0.2741 - val_loss: 0.2825
Epoch 1384/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8754 - disc_loss: 0.1699 - rmse: 0.3070 - val_loss: 0.3116
Epoch 1385/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2902 - disc_loss: 0.1522 - rmse: 0.2651 - val_loss: 0.2754
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5739 - disc_loss: 0.1606 - rmse: 0.3015 - val_loss: 0.2808
Epoch 1387/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.2727 - disc_loss: 0.1676 - rmse: 0.2103 - val_loss: 0.3089
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5139 - disc_loss: 0.1580 - rmse: 0.2835 - val_loss: 0.2788
Epoch 1389/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5805 - disc_loss: 0.1561 - rmse: 0.2401 - val_loss: 0.2566
Epoch 1390/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [16]:
print(real_df_all.shape)
rnn_target_column = 'do'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(51288, 48)
do


In [17]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (35901, 48) val_df.shape :  (10258, 48) test_df.shape: (5129, 48)


In [18]:
#fake_df.shape[0]/8760

In [19]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [22]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  2
out_num_features :  1


In [23]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [24]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/do/


In [25]:
rnn_epochs

20

## 모델 학습

In [26]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.7842 - mean_absolute_error: 0.7199 - nse: 0.2021
Epoch 00001: val_loss improved from inf to 0.70051, saving model to save/best_model.h5
10/10 [==============================] - 9s 882ms/step - loss: 0.7842 - mean_absolute_error: 0.7199 - nse: 0.2021 - val_loss: 0.7005 - val_mean_absolute_error: 0.6681 - val_nse: 0.4127
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.4702 - mean_absolute_error: 0.5325 - nse: 0.5097
Epoch 00002: val_loss improved from 0.70051 to 0.48857, saving model to save/best_model.h5
10/10 [==============================] - 8s 844ms/step - loss: 0.4702 - mean_absolute_error: 0.5325 - nse: 0.5097 - val_loss: 0.4886 - val_mean_absolute_error: 0.5490 - val_nse: 0.5900
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.4043 - mean_absolute_error: 0.4829 - nse: 0.5644
Epoch 00003: val_loss improved from 0.48857 to 0.46974, saving model to save/best_model.h5
10/1

10/10 [==============================] - ETA: 0s - loss: 0.2629 - mean_absolute_error: 0.3823 - nse: 0.7239
Epoch 00022: val_loss did not improve from 0.37431
10/10 [==============================] - 9s 943ms/step - loss: 0.2629 - mean_absolute_error: 0.3823 - nse: 0.7239 - val_loss: 0.5493 - val_mean_absolute_error: 0.5918 - val_nse: 0.5395
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.2601 - mean_absolute_error: 0.3811 - nse: 0.7352
Epoch 00023: val_loss did not improve from 0.37431
10/10 [==============================] - 9s 926ms/step - loss: 0.2601 - mean_absolute_error: 0.3811 - nse: 0.7352 - val_loss: 0.5148 - val_mean_absolute_error: 0.5708 - val_nse: 0.5668
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.2595 - mean_absolute_error: 0.3828 - nse: 0.7298
Epoch 00024: val_loss did not improve from 0.37431
10/10 [==============================] - 9s 927ms/step - loss: 0.2595 - mean_absolute_error: 0.3828 - nse: 0.7298 - val_loss: 0.5

10/10 [==============================] - ETA: 0s - loss: 0.1574 - mean_absolute_error: 0.2987 - nse: 0.8373
Epoch 00045: val_loss did not improve from 0.37431
10/10 [==============================] - 8s 846ms/step - loss: 0.1574 - mean_absolute_error: 0.2987 - nse: 0.8373 - val_loss: 0.9843 - val_mean_absolute_error: 0.7950 - val_nse: 0.1707
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.1680 - mean_absolute_error: 0.3067 - nse: 0.8207
Epoch 00046: val_loss did not improve from 0.37431
10/10 [==============================] - 8s 831ms/step - loss: 0.1680 - mean_absolute_error: 0.3067 - nse: 0.8207 - val_loss: 1.0104 - val_mean_absolute_error: 0.8101 - val_nse: 0.1522
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.1640 - mean_absolute_error: 0.3047 - nse: 0.8173
Epoch 00047: val_loss did not improve from 0.37431
10/10 [==============================] - 8s 838ms/step - loss: 0.1640 - mean_absolute_error: 0.3047 - nse: 0.8173 - val_loss: 1.0

## core / window.py / 

In [27]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  9.776306
predict_day :  5
------------------------
1.5873889 4.1860924 2.0459943
nse3 :  [0.62625897]
pbias3 :  [9.693725]
save/yeong/models/do/
year : 2015 ~ 2020
run :  range(0, 1)
target :  do
length :  51288
train=7, test=1, val=2
---------------
[0.62625897]
[9.693725]


In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
